In [ ]:
from google.colab import files
uploads=files.upload()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("Resume.csv")

print(f"Dataset shape: {df.shape}")

print("\nFirst few rows:")
print(df.head())

print("\nColumns:")
print(df.columns)

In [ ]:
print("="*70)
print("RESUME DATASET EXPLORATION")
print("="*70)

print(f"\nTotal resumes: {len(df):,}")
print(f"\nColumns: {df.columns.tolist()}")

# Check categories (job roles)
print("\n" + "="*70)
print("RESUME CATEGORIES (Job Roles)")
print("="*70)
print(df['Category'].value_counts())

print(f"\nTotal categories: {df['Category'].nunique()}")

# Visualize
plt.figure(figsize=(12, 6))
df['Category'].value_counts().plot(kind='bar', color='skyblue')
plt.title('Resume Distribution by Job Category')
plt.xlabel('Category')
plt.ylabel('Number of Resumes')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Sample resume text
print("\n" + "="*70)
print("SAMPLE RESUME")
print("="*70)
print(f"\nCategory: {df['Category'].iloc[0]}")
print(f"\nResume Text (first 300 chars):")
print(df['Resume_str'].iloc[0][:300])

print(f"\n\nAnother sample - Category: {df['Category'].iloc[10]}")
print(f"\nResume Text (first 300 chars):")
print(df['Resume_str'].iloc[10][:300])

In [ ]:
!pip install beautifulsoup4 -q

import re
import string
from bs4 import BeautifulSoup

print("="*70)
print("STEP 2: TEXT PREPROCESSING")
print("="*70)

# Function to clean resume text
def clean_resume_text(text):
    # Remove HTML tags (some resumes have HTML)
    text = BeautifulSoup(text, "html.parser").get_text()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)

    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)

    # Remove special characters and digits (keep letters and spaces)
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)

    # Convert to lowercase
    text = text.lower()

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

# Apply cleaning to all resumes
print("\nCleaning resumes...")
df['Resume_clean'] = df['Resume_str'].apply(clean_resume_text)

print("✓ Cleaning complete!")

# Show before/after examples
print("\n" + "="*70)
print("BEFORE/AFTER CLEANING")
print("="*70)

print("\n--- Example 1 ---")
print(f"Category: {df['Category'].iloc[0]}")
print(f"\nBEFORE (first 200 chars):")
print(df['Resume_str'].iloc[0][:200])
print(f"\nAFTER (first 200 chars):")
print(df['Resume_clean'].iloc[0][:200])

print("\n--- Example 2 ---")
print(f"Category: {df['Category'].iloc[50]}")
print(f"\nBEFORE (first 200 chars):")
print(df['Resume_str'].iloc[50][:200])
print(f"\nAFTER (first 200 chars):")
print(df['Resume_clean'].iloc[50][:200])

# Check text length distribution
df['text_length'] = df['Resume_clean'].str.len()

print("\n" + "="*70)
print("TEXT LENGTH STATISTICS")
print("="*70)
print(df['text_length'].describe())

print(f"\n✓ Step 2 Complete: {len(df):,} resumes preprocessed")

In [ ]:
print("="*70)
print("STEP 3: SKILL EXTRACTION")
print("="*70)

# Define skill keywords to extract

SKILLS_DATABASE = {
    'Programming': [
        'python', 'java', 'javascript', 'c++', 'c#', 'ruby', 'php', 'swift',
        'kotlin', 'r', 'sql', 'html', 'css', 'typescript', 'go', 'rust'
    ],
    'Data Science & ML': [
        'machine learning', 'deep learning', 'data science', 'data analysis',
        'artificial intelligence', 'neural networks', 'nlp', 'computer vision',
        'data mining', 'predictive modeling', 'statistical analysis'
    ],
    'Tools & Frameworks': [
        'tensorflow', 'pytorch', 'keras', 'scikit-learn', 'pandas', 'numpy',
        'matplotlib', 'seaborn', 'tableau', 'power bi', 'excel', 'spark',
        'hadoop', 'docker', 'kubernetes', 'git', 'aws', 'azure', 'gcp'
    ],
    'Databases': [
        'mysql', 'postgresql', 'mongodb', 'oracle', 'sql server', 'redis',
        'cassandra', 'dynamodb', 'sqlite'
    ],
    'Soft Skills': [
        'leadership', 'communication', 'teamwork', 'problem solving',
        'project management', 'analytical', 'critical thinking', 'agile',
        'scrum', 'collaboration'
    ],
    'Web Development': [
        'react', 'angular', 'vue', 'node.js', 'django', 'flask', 'spring',
        'rest api', 'graphql', 'web development', 'frontend', 'backend'
    ]
}

# Flatten all skills into one list
all_skills = []
for category, skills in SKILLS_DATABASE.items():
    all_skills.extend(skills)

def extract_skills(text):
    """
    Extract skills from resume text
    Returns: list of found skills
    """
    text_lower = text.lower()
    found_skills = []

    for skill in all_skills:
        # Use word boundaries to match whole words/phrases
        pattern = r'\b' + re.escape(skill) + r'\b'
        if re.search(pattern, text_lower):
            found_skills.append(skill)

    return found_skills

# Extract skills from all resumes
print("\nExtracting skills from resumes...")
df['extracted_skills'] = df['Resume_clean'].apply(extract_skills)
df['num_skills'] = df['extracted_skills'].apply(len)

print("✓ Skill extraction complete!")

# Statistics
print("\n" + "="*70)
print("SKILL EXTRACTION STATISTICS")
print("="*70)
print(f"\nAverage skills per resume: {df['num_skills'].mean():.1f}")
print(f"Min skills found: {df['num_skills'].min()}")
print(f"Max skills found: {df['num_skills'].max()}")

print("\nSkills distribution:")
print(df['num_skills'].describe())

# Show examples
print("\n" + "="*70)
print("SAMPLE SKILL EXTRACTION")
print("="*70)

for i in [0, 10, 20]:
    print(f"\n--- Resume {i+1} ---")
    print(f"Category: {df['Category'].iloc[i]}")
    print(f"Skills found ({len(df['extracted_skills'].iloc[i])}): {df['extracted_skills'].iloc[i][:10]}")

print(f"\n✓ Step 3 Complete: Skills extracted from {len(df):,} resumes")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

print("="*70)
print("STEP 4: JOB MATCHING & SCORING LOGIC")
print("="*70)

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(
    max_features=5000,
    stop_words='english',
    ngram_range=(1, 2)
)

# Fit on all resume text
print("\nVectorizing resumes...")
resume_vectors = vectorizer.fit_transform(df['Resume_clean'])
print(f"✓ Created vectors: {resume_vectors.shape}")

# Function to match resumes with job description
def match_resumes_to_job(job_description, top_n=10):
    """
    Match resumes to a job description

    Args:
        job_description: str - The job posting text
        top_n: int - Number of top candidates to return

    Returns:
        DataFrame with ranked candidates
    """
    # Clean job description
    job_clean = clean_resume_text(job_description)

    # Extract required skills from job
    job_skills = extract_skills(job_clean)

    # Vectorize job description
    job_vector = vectorizer.transform([job_clean])

    # Calculate cosine similarity
    similarities = cosine_similarity(job_vector, resume_vectors)[0]

    # Add similarity scores to dataframe
    results_df = df.copy()
    results_df['match_score'] = similarities * 100  # Convert to percentage

    # Calculate skill match
    def calculate_skill_match(candidate_skills):
        if len(job_skills) == 0:
            return 0, [], job_skills
        matching = [s for s in job_skills if s in candidate_skills]
        missing = [s for s in job_skills if s not in candidate_skills]
        match_pct = (len(matching) / len(job_skills)) * 100
        return match_pct, matching, missing

    results_df[['skill_match_pct', 'matching_skills', 'missing_skills']] = \
        results_df['extracted_skills'].apply(
            lambda x: pd.Series(calculate_skill_match(x))
        )

    # Combined score (70% text similarity + 30% skill match)
    results_df['final_score'] = (
        0.7 * results_df['match_score'] +
        0.3 * results_df['skill_match_pct']
    )

    # Sort by score
    results_df = results_df.sort_values('final_score', ascending=False)

    # Return top N with relevant columns
    return results_df[[
        'Category', 'final_score', 'match_score', 'skill_match_pct',
        'num_skills', 'matching_skills', 'missing_skills', 'Resume_clean'
    ]].head(top_n), job_skills

print("\n✓ Matching system ready!")

# Test with a sample job description
print("\n" + "="*70)
print("TEST: DATA SCIENTIST JOB")
print("="*70)

job_desc = """
Data Scientist Position

We are looking for a Data Scientist with strong Python and Machine Learning skills.

Requirements:
- Python programming
- Machine Learning experience
- Data analysis and statistical analysis
- SQL and databases
- Experience with pandas, scikit-learn
- Communication and teamwork skills

Preferred:
- Deep learning
- TensorFlow or PyTorch
- AWS or cloud experience
"""

print("\nJob Description:")
print(job_desc[:200] + "...")

print("\nMatching resumes...")
top_candidates, required_skills = match_resumes_to_job(job_desc, top_n=10)

print(f"\n✓ Found top 10 candidates!")
print(f"Required skills: {required_skills}")

print("\n" + "="*70)
print("TOP 5 CANDIDATES")
print("="*70)

for idx, row in top_candidates.head(5).iterrows():
    print(f"\n--- Candidate {list(top_candidates.index).index(idx) + 1} ---")
    print(f"Category: {row['Category']}")
    print(f"Final Score: {row['final_score']:.2f}%")
    print(f"  - Text Match: {row['match_score']:.2f}%")
    print(f"  - Skill Match: {row['skill_match_pct']:.2f}%")
    print(f"Matching Skills: {row['matching_skills']}")
    print(f"Missing Skills: {row['missing_skills']}")

print(f"\n✓ Step 4 Complete: Matching & scoring system built")

In [ ]:
print("="*70)
print("STEP 5: CANDIDATE RANKING WITH EXPLANATIONS")
print("="*70)

def explain_candidate_ranking(candidate_row, rank, total_required_skills):
    """
    Generate human-readable explanation for non-technical users
    """
    explanation = f"""
--------------------------------------------------------------------
 CANDIDATE #{rank}
--------------------------------------------------------------------

📋 CATEGORY: {candidate_row['Category']}

🎯 OVERALL FIT: {candidate_row['final_score']:.1f}% Match

📊 BREAKDOWN:
   • Resume Relevance: {candidate_row['match_score']:.1f}%
   • Skills Match: {candidate_row['skill_match_pct']:.1f}% ({len(candidate_row['matching_skills'])}/{total_required_skills} required skills)

✅ MATCHING SKILLS ({len(candidate_row['matching_skills'])}):
   {', '.join(candidate_row['matching_skills']) if candidate_row['matching_skills'] else 'None'}

❌ MISSING SKILLS ({len(candidate_row['missing_skills'])}):
   {', '.join(candidate_row['missing_skills']) if candidate_row['missing_skills'] else 'None'}

💡 RECOMMENDATION:
   {'STRONG MATCH - Interview recommended' if candidate_row['final_score'] >= 50
    else 'GOOD MATCH - Consider for interview' if candidate_row['final_score'] >= 35
    else 'PARTIAL MATCH - May need training' if candidate_row['final_score'] >= 20
    else 'WEAK MATCH - Not recommended'}

📝 KEY INSIGHT:
   {f"Candidate has {len(candidate_row['matching_skills'])}/{total_required_skills} required skills. " +
    (f"Strong in: {', '.join(candidate_row['matching_skills'][:3])}" if len(candidate_row['matching_skills']) > 0 else "Limited relevant experience") +
    (f". Needs development in: {', '.join(candidate_row['missing_skills'][:3])}" if len(candidate_row['missing_skills']) > 0 else "")}

--------------------------------------------------------
    """
    return explanation

# Show detailed explanations for top 3
print("\n" + "="*70)
print("DETAILED CANDIDATE EXPLANATIONS (FOR HR/MANAGERS)")
print("="*70)

for i, (idx, row) in enumerate(top_candidates.head(3).iterrows(), 1):
    print(explain_candidate_ranking(row, i, len(required_skills)))

# Create summary table
print("\n" + "="*70)
print("RANKING SUMMARY TABLE")
print("="*70)

summary = top_candidates.head(10)[['Category', 'final_score', 'skill_match_pct']].copy()
summary.columns = ['Job Category', 'Overall Match %', 'Skills Match %']
summary['Rank'] = range(1, len(summary) + 1)
summary = summary[['Rank', 'Job Category', 'Overall Match %', 'Skills Match %']]

print(summary.to_string(index=False))

print(f"\n✓ Step 5 Complete: Rankings explained clearly for non-technical users")

In [ ]:
import joblib

print("="*70)
print("SAVING MODELS FOR DEPLOYMENT")
print("="*70)

# Save the vectorizer
joblib.dump(vectorizer, 'resume_vectorizer.pkl')
print("✓ Vectorizer saved")

# Save resume vectors
joblib.dump(resume_vectors, 'resume_vectors.pkl')
print("✓ Resume vectors saved")

# Save the processed dataframe (with cleaned text and skills)
df.to_csv('processed_resumes.csv', index=False)
print("✓ Processed resumes saved")

# Save skills database
joblib.dump(SKILLS_DATABASE, 'skills_database.pkl')
print("✓ Skills database saved")

# Save all skills list
joblib.dump(all_skills, 'all_skills.pkl')
print("✓ All skills list saved")


print("\nFiles created:")
print("1. resume_vectorizer.pkl")
print("2. resume_vectors.pkl")
print("3. processed_resumes.csv")
print("4. skills_database.pkl")
print("5. all_skills.pkl")

# Download if in Colab
try:
    from google.colab import files
    files.download('resume_vectorizer.pkl')
    files.download('resume_vectors.pkl')
    files.download('processed_resumes.csv')
    files.download('skills_database.pkl')
    files.download('all_skills.pkl')
    print("\n✓ Files downloaded!")
except:
    print("\n✓ Files saved locally!")